# Projet Capsule ISI CO_05 Pierre Bourgueil

J'ai choisi de ne pas utiliser de jeu de données externe mais de générer moi-même mes données d'entrainement grâce à des simulations encadrées par des règles.

J'ai toujours été fasciné par les IA qui apprennent à jouer à des jeux en s'entrainant contre elles-mêmes et j'ai voulu faire ça moi aussi.
Le jeu que j'ai choisi est le mastermind.


![Image_jeu_Mastermind](https://upload.wikimedia.org/wikipedia/commons/2/2d/Mastermind.jpg)

## Règle du jeu

Le jeu se joue à deux : un codificateur et un décodeur.
Le but est de deviner, par déductions successives, la couleur de chacun des 4 pions cachés derrière un écran.
Les couleurs possibles sont au nombre de 6 (rouge, vert, bleu, jaune, orange, violet).

Déroulement du jeu : le codificateur pose à l'endroit prévu à cet effet, les 4 pions de son choix. Il doit prendre soin de ne pas révéler la couleur et la répartition dans les trous des pions. Rien ne l'empêche d'en choisir plusieurs d'une même couleur.

Son adversaire, le décodeur, est chargé de déchiffrer ce code secret. Il doit le faire en 10 coups maximum. Il place 4 ou 5 pions dans les trous de la première rangée immédiatement près de lui. Si l'un des pions correspond par sa position et sa couleur à un pion caché derrière l'écran, le codificateur l'indique en plaçant une fiche noire dans l'un des trous de marque, sur le côté droit correspondant du plateau. Si l'un des pions correspond uniquement par sa couleur, le Codificateur l'indique par une fiche blanche dans l'un des trous de marque. S'il n'y a aucune correspondance, il ne marque rien.

Le décodeur continue de poser des pions par rangées successives. La tactique consiste à sélectionner en fonction des coups précédents, couleurs et positions, de manière à obtenir le maximum d'informations de la réponse du partenaire puisque le nombre de propositions est limité par le nombre de rangées de trous du jeu. Dans la plupart des cas, il s'efforce de se rapprocher le plus possible de la solution, compte tenu des réponses précédentes, mais il peut aussi former une combinaison dans le seul but de vérifier une partie des conclusions des coups précédents et de faire en conséquence la proposition la plus propice à la déduction d'une nouvelle information.

Quand il est arrivé, au bout d'un certain nombre de coups, à placer les 4 pions qui correspondent exactement par la couleur et la position à ceux du code, la manche est terminée.


## Approche choisie

Pour des raisons de curiosité, de performances, de temps et de complexité, j’ai choisi de ne pas m’orienter vers le Reinforcement Learning.
L’approche choisie est le **Monte Carlo Tree Search (MCTS)**

En informatique, et plus précisément en intelligence artificielle, la recherche arborescente Monte Carlo ou Monte Carlo tree search (MCTS) est un algorithme de recherche heuristique utilisé dans le cadre de la prise de décision. Il est notamment employé dans les jeux. On peut citer son implémentation dans le jeu vidéo Total War: Rome II avec son mode campagne IA haut-niveau1 et les récents programmes informatiques de Go2, suivis par les échecs et shogi3, ainsi que les jeux vidéo en temps réel et les jeux à information incomplète tels que le poker4.

![Image_jeu_Mastermind](https://upload.wikimedia.org/wikipedia/commons/thumb/0/04/MCTS.svg/1212px-MCTS.svg.png)


L'algorithme MCTS est un algorithme qui explore l'arbre des possibles. La racine est la configuration initiale du jeu. Chaque nœud est une configuration et ses enfants sont les configurations suivantes. MCTS conserve en mémoire un arbre qui correspond aux nœuds déjà explorés de l'arbre des possibles. Une feuille de cet arbre (un nœud n'ayant pas d'enfants) est soit une configuration finale (i.e. on sait si un des joueurs a gagné, ou s'il y a match nul), soit un nœud dont aucun enfant n'a encore été exploré. Dans chaque nœud, on stocke deux nombres : le nombre de simulations gagnantes, et le nombre total de simulations. Chaque étape est composée de quatre phases:

1. *Sélection*: Depuis la racine, on sélectionne successivement des enfants jusqu'à atteindre une feuille. Dans cette phase, le choix des enfants est guidé par un compromis entre exploitation (aller vers un enfant qui a été prouvé comme prometteur) et exploration (aller visiter un autre enfant, qui a l'air moins prometteur mais qui pourrait l'être davantage). Dans l'exemple donné dans la figure, on choisit la feuille de gauche (de profondeur 3).
2. *Expansion*: si cette feuille n'est pas finale, créer un enfant (ou plusieurs) en utilisant les règles du jeu et choisir l'un des enfants. Sur l'exemple, on ajoute cet enfant, marqué 0/0.
3. *Simulation*: simuler une exécution d'une partie au hasard depuis cet enfant, jusqu'à atteindre une configuration finale.
4. *Rétropropagation* (Backpropagation): utiliser le résultat de la partie au hasard et mettre à jour les informations sur la branche en partant du nœud enfant vers la racine. Sur l'exemple, la partie était perdante pour blanc. On incrémente donc uniquement le nombre de simulations totales sur la branche pour les nœuds blancs et on incrémente le nombre de simulations gagnées et le nombre de simulations totales pour les noirs sur la branche : 0/0 devient 0/1, 3/3 devient 3/4, 5/6 devient 5/7, 7/10 devient 7/11, et 12/21 devient 12/22. Si la partie avait été gagnante :0/0 serait devenu 1/1, 3/3 serait devenu 4/4, 5/6 serait devenu 6/7, 7/10 serait devenu 8/11, et 12/21 serait devenu 13/22.


In [1]:
import random
import numpy as np
import statistics
from itertools import product
from matplotlib import pyplot as plt
from scipy.stats import norm

La fonction evaluate_guess permet de comparer un code secret et un code proposé par le joueur. Elle renvoie le nombre de pions bien placés et le nombre de pions mal placés.

In [2]:
def redlog(message):
    print(f"\033[91m{message}\033[0m")

def bluelog(message):
    print(f"\033[94m{message}\033[0m")


def evaluate_guess(guess, secret_code):
    exact = sum([1 for i in range(len(secret_code)) if guess[i] == secret_code[i]])
    return exact, sum([min(guess.count(j), secret_code.count(j)) for j in set(guess)]) - exact

Voici notre arcbre de probabilité sous la forme de 2 types de noeuds : les noeud Guess et en réponse les noeuds Feedback.

In [3]:
class GuessNode:
    def __init__(self, parent, parameters, guess):
        self.parameters = parameters
        self.guess = guess

        self.children = []
        self.parent = parent

        self.possible_codes = parent.possible_codes.copy()
        self.history = parent.history.copy()
        self.moves = parent.moves + 1  # (node depth)

        self.total_moves = 0  # sum of moves of all children (for ucb1)
        self.visits = 0

    def expand(self):  # (add feedback children)
        if len(self.possible_codes) == 1:
            redlog("ERROR: extent of a terminal GuessNode")

        possible_feedbacks = {}

        for code in self.possible_codes:
            feedback = evaluate_guess(code, self.guess)

            if feedback not in possible_feedbacks:
                possible_feedbacks[feedback] = [code]
            else:
                possible_feedbacks[feedback].append(code)

        n = len(self.possible_codes)

        for feedback in possible_feedbacks:
            child = FeedbackNode(self.parameters, feedback=feedback, parent=self,
                                 frequency=len(possible_feedbacks[feedback]) / n)
            self.children.append(child)

    def update(self, n_moves):
        self.visits += 1
        self.total_moves += n_moves

    def score(self):  # (ucb1)
        if self.visits == 0:
            return float('inf')
        score = -self.total_moves / self.visits + np.sqrt(2 * np.log(self.parent.visits) / self.visits)
        return score

    def simulate(self):
        moves, possible_codes, guess = self.moves, self.possible_codes.copy(), self.guess
        secret_code = random.choice(self.possible_codes)
        while True:
            feedback = evaluate_guess(guess, secret_code)
            if feedback == (self.parameters["code_length"], 0):
                return moves
            new_possible_codes = []
            for code in possible_codes:
                if evaluate_guess(guess, code) == feedback:
                    new_possible_codes.append(code)
            possible_codes = new_possible_codes
            moves += 1
            guess = random.choice(possible_codes)

    def __str__(self):
        return f"GuessNode : {self.history} With guess: {self.guess}"

    def is_terminal(self):
        return len(self.possible_codes) == 1


class FeedbackNode:
    def __init__(self, parameters, feedback=None, parent=None, frequency=1.):
        self.parameters = parameters
        self.feedback = feedback
        self.frequency = frequency

        self.visits = 0

        if parent:
            self.moves = parent.moves
            self.guess = parent.guess
            self.history = parent.history.copy()
            self.history.append((parent.guess, feedback))
            self.possible_codes = [code for code in parent.possible_codes if
                                   evaluate_guess(self.guess, secret_code=code) == feedback]
        else:
            self.moves = 0
            self.guess = None
            self.history = []
            self.possible_codes = list(
                product(range(1, self.parameters["num_colors"] + 1), repeat=self.parameters["code_length"]))

        self.parent = parent
        self.children = []

        self.expand()

    def simulate(self):
        redlog("ERROR: FeedbackNode should not simulate")

    def expand(self):  # (add_guess_children)
        for code in self.possible_codes:
            child = GuessNode(self, self.parameters, code)
            self.children.append(child)

    def score(self):
        return self.frequency

    def update(self, moves):
        self.visits += 1

    def __str__(self):
        return f"FeedbackNode : {self.history}"

    def is_terminal(self):
        redlog("ERROR: GuessNode should have been caught first")
        return len(self.possible_codes) == 1


La fonction select (phase 1/4 de MCTS) qui permet de choisir le noeud à explorer en fonction de l'UCB1.

In [4]:
def select(node):
    guess_nodes = node.children
    if not guess_nodes:
        return node
    chosen_guess_node = max(guess_nodes, key=lambda guess_node: guess_node.score())

    if chosen_guess_node.children:
        feedback_nodes = chosen_guess_node.children
        probas = [node.score() for node in feedback_nodes]
        chosen_feedback_node = np.random.choice(feedback_nodes, p=probas)
        return select(chosen_feedback_node)

    return chosen_guess_node

La fonction backpropagate (phase 4/4 de MCTS) qui permet de mettre à jour les informations des noeuds parents en fonction des résultats des simulations.

In [5]:
def backpropagate(node, total_moves):
    while node is not None:
        node.update(total_moves)
        node = node.parent

Les autres phases sont gérées par les fonctions expand et simulate des classes GuessNode et FeedbackNode.

Notre fonction principale mcts_training qui va entrainer notre Arbre.

In [2]:
def mcts_training(node, num_iterations):
    for i in range(num_iterations+1):
        if i % 10 == 0:
            bluelog(f"\nITERATION {i}/{num_iterations}")
        guess_node = select(node)
        if guess_node.is_terminal():
            total_moves = guess_node.moves

        else:
            guess_node.expand()
            total_moves = guess_node.simulate()
        backpropagate(guess_node, total_moves)
    return node

Testons avec une version très légère du jeu: 2 possibilités pout 3 couleurs.

In [3]:
parameters = {
    "code_length": 2,
    "num_colors": 3
}
root = FeedbackNode(parameters)
mcts_training(root, 10000)

NameError: name 'FeedbackNode' is not defined

Essayons de représenter cet arbre

In [11]:
def show_f(node_f, depth=1):
    print(f"{'  ' * depth}{node_f} ({node_f.visits} visits, frequency:{node_f.frequency})")
    for child in node_f.children:
        show_g(child, depth + 1)

def show_g(node_g, depth=1):
    print(f"{'  ' * depth}{node_g} ({node_g.visits} visits, {node_g.total_moves} total_moves)")
    for child in node_g.children:
        show_f(child, depth + 1)

show_f(root)

  FeedbackNode : [] (10001 visits, frequency:1.0)
    GuessNode : [] With guess: (1, 1) (110 visits, 329 total_moves)
      FeedbackNode : [((1, 1), (2, 0))] (13 visits, frequency:0.1111111111111111)
        GuessNode : [((1, 1), (2, 0))] With guess: (1, 1) (13 visits, 26 total_moves)
      FeedbackNode : [((1, 1), (1, 0))] (50 visits, frequency:0.4444444444444444)
        GuessNode : [((1, 1), (1, 0))] With guess: (1, 2) (12 visits, 36 total_moves)
          FeedbackNode : [((1, 1), (1, 0)), ((1, 2), (2, 0))] (6 visits, frequency:0.25)
            GuessNode : [((1, 1), (1, 0)), ((1, 2), (2, 0))] With guess: (1, 2) (6 visits, 18 total_moves)
          FeedbackNode : [((1, 1), (1, 0)), ((1, 2), (1, 0))] (2 visits, frequency:0.25)
            GuessNode : [((1, 1), (1, 0)), ((1, 2), (1, 0))] With guess: (1, 3) (2 visits, 6 total_moves)
          FeedbackNode : [((1, 1), (1, 0)), ((1, 2), (0, 2))] (2 visits, frequency:0.25)
            GuessNode : [((1, 1), (1, 0)), ((1, 2), (0, 2))] With 

On peut déjà voir à quel point l'arbre est grand. On voit aussi très bien l'alternance des noeuds Guess et Feedback. on voit aussi que certains noeuds n'ont pas été visités. (visits: 0)

Essayons d'évaluer les performances de notre arbre.

Tout d'abord, que donne une stratégie random au mastermind ?

In [4]:
def play_random(parameters, n=1000):
    def game():
        secret_code = random.choices(range(1, parameters['num_colors'] + 1), k=parameters['code_length'])
        possible_codes = list(product(range(1, parameters["num_colors"] + 1), repeat=parameters["code_length"]))
        attempts = 0
        while True:
            guess = random.choice(possible_codes)
            feedback = evaluate_guess(guess, secret_code)
            attempts += 1
            if feedback == (parameters["code_length"], 0):
                return attempts
            possible_codes = [code for code in possible_codes if evaluate_guess(guess, code) == feedback]

    return [game() for _ in range(n)]


games_random = play_random({"code_length": 2, "num_colors": 3}, 1000)

NameError: name 'random' is not defined

In [5]:
def plot_strategy_performance(games):
    count, bins, ignored = plt.hist(games, bins=range(1, 13), align='left', rwidth=0.8, color='skyblue',
                                    edgecolor='black')

    mean = np.mean(games)
    std = np.std(games)
    xmin, xmax = plt.xlim()
    x = np.linspace(xmin, xmax, 300)
    p = norm.pdf(x, mean, std) * len(games) * (bins[1] - bins[0])

    plt.plot(x, p, 'k', linewidth=2)

    title = "Fit results: mean = %.2f,  std = %.2f" % (mean, std)
    plt.title(title)
    plt.xlabel('Attempts')
    plt.ylabel('Frequency')
    plt.show()

On arrive en moyenne à  coups pour réussir le jeu en choisissant des combinaisons aléatoirement au sein de l'ensemble des combinaisons restantes possibles.
C'est surprenamment performant. Le joueur occasionnel que je suis descend très rarement en dessous des 6 coups pour trouver la bonne combinaison.


Essayons maintenant de jouer avec notre arbre.

In [ ]:
def play_with_mcts(parameters, root, n):
    def game():
        secret_code = random.choices(range(1, parameters['num_colors'] + 1), k=parameters['code_length'])
        node = root
        while True:
            guess_node = max(node.children, key=lambda guess_node: guess_node.score())
            guess = guess_node.guess
            feedback = evaluate_guess(guess, secret_code)
            print(f"Node children: {guess}, guess: {guess}, feedback: {feedback}")


            if feedback == (parameters["code_length"], 0):
                return guess_node.moves
            if not guess_node.children:
                redlog("RETRAINING")
                mcts_training(node, 100, parameters)

            for child in guess_node.children:
                if child.feedback == feedback:
                    node = child
                    break

    games = []
    for i in range(n):
        print(i)
        games.append(game())
    return games


mcts_games = play_with_mcts(parameters, root, 10)

In [ ]:
plot_strategy_performance(mcts_games)